In [ ]:
import scanpy as sc
import os
import numpy as np
import pandas as pd
import sys
import seaborn as sns
import matplotlib.pyplot as plt
import pylab as pl

In [ ]:
data_path = "../../data"

In [ ]:
ad_list = []
for adata_name in ["mock", "4hr_avr", "6hr_avr", "9hr_avr", "avrrpt24"]:
    ad_sp_only = sc.read(
        os.path.join(data_path, "integration", adata_name, "spatial_data.h5ad")
    )
    ad_list.append(ad_sp_only)
ad_sp_only = sc.concat(ad_list)

In [ ]:
output_folder = r"./"

In [ ]:
figure_output_folder = "6ac"
try:
    os.mkdir(os.path.join(output_folder, figure_output_folder))
except:
    pass

In [ ]:
sc.set_figure_params(dpi=300, dpi_save=400)

In [ ]:
try:
    os.mkdir(os.path.join(figure_output_folder, "figures"))
except:
    print("Folder already exists")

In [ ]:
ad_sp = ad_sp_only.copy()

In [ ]:
immune_active = {"9", "11", "7", "19", "10", "15", "12", "8"}
ad_sp.obs["predicted_immune_categorized_celltype"] = [
    (
        "Immune activated " + ad_sp.obs["celltype"].values[p]
        if (
            (ad_sp.obs["SCT_snn_res.1"].values[p] in immune_active)
            & (ad_sp.obs["celltype"].values[p] != "Vasculature")
            & (ad_sp.obs["celltype"].values[p] != "Epidermis")
        )
        else ad_sp.obs["celltype"].values[p]
    )
    for p in range(len(ad_sp.obs.index))
]

In [ ]:
pd.crosstab(ad_sp.obs["predicted_immune_categorized_celltype"], ad_sp.obs["batch"])

In [ ]:
ad_sp = ad_sp[~ad_sp.obs["celltype"].isin(["Unknown"])]

In [ ]:
df = pd.DataFrame(
    ad_sp.obs["predicted_immune_categorized_celltype"].values,
    index=ad_sp.obs["leiden"].values,
    columns=["leiden"],
)
df = pd.crosstab(df.index, df.leiden)

max_index = np.argmax(df.values, axis=1)

# Create a new DataFrame with the max_index as a sorting key
sorted_df = df.iloc[max_index.argsort()]
plt.figure()
# Create the clustermap with the sorted DataFrame
g = sns.clustermap(
    sorted_df, standard_scale=0, col_cluster=False, row_cluster=False, cmap="Blues"
)
plt.setp(g.ax_heatmap.yaxis.get_majorticklabels(), rotation=0)
g.ax_heatmap.grid(False)
plt.grid(False)
plt.savefig(os.path.join(figure_output_folder, "figures", "6a.pdf"))
plt.show()

### Sub 6a

In [ ]:
try:
    os.mkdir(os.path.join(figure_output_folder, "figures", "supplemental_6a"))
except:
    None

for adata_name in ["mock", "4hr_avr", "6hr_avr", "9hr_avr", "avrrpt24"]:
    ad_sp_only = sc.read(
        os.path.join(data_path, "integration", adata_name, "spatial_data.h5ad")
    )
    ad_sp = ad_sp_only.copy()
    immune_active = {"9", "11", "7", "19", "10", "15", "12", "8"}
    ad_sp.obs["predicted_immune_categorized_celltype"] = [
        (
            "Immune activated " + ad_sp.obs["celltype"].values[p]
            if (
                (ad_sp.obs["SCT_snn_res.1"].values[p] in immune_active)
                & (ad_sp.obs["celltype"].values[p] != "Vasculature")
                & (ad_sp.obs["celltype"].values[p] != "Epidermis")
            )
            else ad_sp.obs["celltype"].values[p]
        )
        for p in range(len(ad_sp.obs.index))
    ]
    ad_sp = ad_sp[~ad_sp.obs["celltype"].isin(["Unknown"])]
    df = pd.DataFrame(
        ad_sp.obs["predicted_immune_categorized_celltype"].values,
        index=ad_sp.obs["leiden"].values,
        columns=["leiden"],
    )
    df = pd.crosstab(df.index, df.leiden)

    max_index = np.argmax(df.values, axis=1)

    # Create a new DataFrame with the max_index as a sorting key
    sorted_df = df.iloc[max_index.argsort()]
    plt.figure()
    # Create the clustermap with the sorted DataFrame
    g = sns.clustermap(
        sorted_df, z_score=0, col_cluster=False, row_cluster=False, cmap="Blues"
    )
    plt.setp(g.ax_heatmap.yaxis.get_majorticklabels(), rotation=0)
    g.ax_heatmap.grid(False)
    plt.grid(False)
    plt.title(adata_name)
    plt.savefig(
        os.path.join(
            figure_output_folder, "figures", "supplemental_6a", f"{adata_name}.pdf"
        )
    )
    plt.show()